In [1]:
import omnipath
from omnipath._core.utils._homologene import download_homologene, show_homologene

/home/dbdimitrov/anaconda3/envs/spiana/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
import liana as li

In [3]:
import pandas as pd
import numpy as np
from itertools import product

Simplify fun, to be done by a single-column at a time

In [4]:
import pandas as pd

In [5]:
# define a function to replace list elements with dictionary values
def _replace_subunits(lst, my_dict, one_to_many=False):
    result = []
    for x in lst:
        if x in my_dict:
            value = my_dict[x]
            
            if not isinstance(value, list):
                value = [value]
            
            # TODO: one_to_many as integer
            if (not one_to_many) & (len(my_dict[x]) > 1):
                result.append(np.nan)
            else:
                result.append(value)
        else:
            result.append(np.nan)
    return result

In [6]:
def generate_orthologs(data, column, map_dict, one_to_many=False):
    df = data[[column]].drop_duplicates().set_index(column)
    
    df['subunits'] = df.index.str.split('_')
    df['subunits'] = df['subunits'].apply(_replace_subunits, args=(map_dict, one_to_many,))
    df = df['subunits'].explode().reset_index()
    
    grouped = df.groupby(column).filter(lambda x: x['subunits'].notna().all()).groupby(column)
    
    # Generate all possible subunit combinations within each group
    complexes = []
    for name, group in grouped:
        if group['subunits'].isnull().all():
            continue
        subunit_lists = [list(x) for x in group['subunits']]
        complex_combinations = list(product(*subunit_lists))
        for complex in complex_combinations:
            complexes.append((name, '_'.join(complex)))
    
    # Create output DataFrame
    result = pd.DataFrame(complexes, columns=['source', 'target']).set_index('source')
    
    return result

In [ ]:
# map_df = download_homologene('9606', '10090').set_index('source')
# to_many = map_df[map_df.groupby('source').size() >= 2].index

In [7]:
# # one_to_many_complexes
complex1tomany = ['CSF2RA_CSF2RB',
 'CSF2RB_IL5RA',
 'CSF2RB_IL3RA',
 'IL11RA_IL6ST',
 'IFNL3_IFNLR1_IL10RB',
 'IL22_IL22RA1']

In [14]:
data = li.resource.select_resource()
# data = data[data['receptor'].str.contains('_')]
data = data[data['receptor'].isin(complex1tomany)]

In [15]:
column = 'receptor'

In [16]:
data

,ligand,receptor
1841,CSF2,CSF2RA_CSF2RB
1898,IL5,CSF2RB_IL5RA
1975,IL3,CSF2RB_IL3RA
2331,IL11,IL11RA_IL6ST
2992,IFNL3,IFNL3_IFNLR1_IL10RB
3263,IL22,IL22_IL22RA1


In [11]:
# exploded = li.resource.explode_complexes(data)
# list(exploded[exploded['receptor'].isin(to_many)]['receptor_complex'])

SignaLink

In [ ]:
import omnipath as op

In [ ]:
data = op.requests.Annotations.get(resources='SignaLink_pathway', genesymbols=True)

In [ ]:
CPLEX_PREFIX = 'COMPLEX:'

In [ ]:
data['genesymbol'] = data['genesymbol'].replace(CPLEX_PREFIX, '', regex=True)

In [ ]:
column = 'genesymbol'

In [ ]:
# data = data[data[column].str.contains('ABI1_')]
data

Translate

In [12]:
def orthology_translate_column(data, column, id_type, target_organism, replace=True, keep_untranslated=False, source_organism=9606, one_to_many=False):
    # get orthologs
    source_organism, target_organism = str(source_organism), str(target_organism)
    map_df = download_homologene(source_organism, target_organism, id_type).set_index('source')
    map_dict = map_df.groupby(level=0)["target"].apply(list).to_dict()
    map_data = generate_orthologs(data, column, map_dict, one_to_many)
    
    # join orthologs
    data = (data.set_index(column).
            merge(map_data, left_index=True, right_index=True, how='left').
            reset_index(names=column)
            )
    
    # replace orthologs
    if replace:
        data[column] = data['target']
        data = data.dropna(subset=[column]).drop('target', axis=1)
    elif keep_untranslated:
        data['target'] = data.apply(lambda x: x['target'] if not pd.isnull(x['target']) else x[column], axis=1)
    
    return data

In [20]:
orthology_translate_column(data, column, 'genesymbol', 10090, replace=False, keep_untranslated=True, one_to_many=True)

,receptor,ligand,target
0,CSF2RA_CSF2RB,CSF2,Csf2ra_Csf2rb
1,CSF2RA_CSF2RB,CSF2,Csf2ra_Csf2rb2
2,CSF2RB_IL3RA,IL3,Csf2rb_Il3ra
3,CSF2RB_IL3RA,IL3,Csf2rb2_Il3ra
4,CSF2RB_IL5RA,IL5,Csf2rb_Il5ra
5,CSF2RB_IL5RA,IL5,Csf2rb2_Il5ra
6,IFNL3_IFNLR1_IL10RB,IFNL3,Ifnl2_Ifnlr1_Il10rb
7,IFNL3_IFNLR1_IL10RB,IFNL3,Ifnl3_Ifnlr1_Il10rb
8,IL11RA_IL6ST,IL11,Gm13305_Il6st
9,IL11RA_IL6ST,IL11,Il11ra1_Il6st


In [ ]:
replace = False
keep_untranslated = True
one_to_many = False

In [ ]:
map_df = download_homologene('9606', '10090').set_index('source')

In [ ]:
map_dict = map_df.groupby(level=0)["target"].apply(list).to_dict()
map_data = generate_orthologs(data, column, 'subunits', one_to_many).sort_values('source')

In [ ]:
map_data.head()

In [ ]:
data = data.set_index(column).merge(map_data, left_index=True, right_index=True, how='left').reset_index(names=column)

In [ ]:
if replace:
    data[column] = data['target']
    data = data.dropna(subset=[column]).drop('target', axis=1)
elif keep_untranslated:
    data['target'] = data.apply(lambda x: x['target'] if not pd.isnull(x['target']) else x[column], axis=1)

In [ ]:
data

Homology Conversion

In [ ]:
def _explode_complexes(resource: pd.DataFrame,
                       target: str ='source',
                       source: str ='target') -> pd.DataFrame:
    resource['key'] = resource[SOURCE] + '|' + resource[TARGET]
    resource = (resource.set_index('interaction')
                .apply(lambda x: x.str.split('_'))
                .explode([TARGET])
                .explode(SOURCE)
                .reset_index()
                )
    resource[[f'{SOURCE}_complex', f'{TARGET}_complex']] = resource[
        'interaction'].str.split('|', expand=True)

    return resource

In [ ]:
def _rebuild_complexes(df, group_cols, target_col):
    df = resource.copy()
    df_grouped = resource.groupby(group_cols)['target'].agg(lambda x: '_'.join(map(str, x))).reset_index()
    df = df.drop('target', axis=1).merge(df_grouped, on=list(np.setdiff1d(group_cols, target_col)), how='inner')
    df = df.drop(f'{target_col}_complex', axis=1).rename({'target':f'{target_col}_complex'}, axis=1)
    
    return df

In [ ]:
map_df = download_homologene('9606', '10090')

In [ ]:
map_df

In [ ]:
resource = li.resource.select_resource()

In [ ]:
resource = li.resource.explode_complexes(resource)
# map_df = dict(zip(map_df['source'], map_df['target']))

In [ ]:
resource

In [ ]:
resource = resource.merge(map_df, left_on='receptor', right_on='source', how='inner').drop(['source'], axis=1)
resource = _rebuild_complexes(resource, ['interaction', 'ligand'], 'receptor')

In [ ]:
resource

In [ ]:
resource = resource.merge(map_df, left_on='ligand', right_on='source', how='inner').drop(['source'], axis=1)
resource = _rebuild_complexes(resource, ['interaction', 'receptor'], 'ligand')

In [ ]:
resource

In [ ]:
# remove duplicates (1:many mappings)
resource = resource.drop_duplicates(['interaction']).drop(['interaction'], axis=1)

In [ ]:
resource

Only keep full complexes

In [ ]:
map_df = download_homologene('9606', '10090')

In [ ]:
resource = li.resource.select_resource()
resource = li.resource.explode_complexes(resource)
resource = resource.merge(map_df, left_on='receptor', right_on='source', how='inner').drop(['source'], axis=1)

In [ ]:
group_cols = ['interaction', 'ligand']
target = 'receptor'

In [ ]:
resource

In [ ]:
df = resource.copy()

df_grouped = resource.groupby(group_cols)['target'].agg(lambda x: '_'.join(map(str, x))).reset_index()

In [ ]:
df_grouped

In [ ]:
# cplex_table['subunits2']